### Test Camel Model

In [1]:
import os, requests
from llm import call_llm
from psi_to_cactus import convert_psi_file_case_id_to_cactus

os.environ["NO_PROXY"] = "127.0.0.1,localhost"
os.environ["no_proxy"] = os.environ["NO_PROXY"]

r = requests.get("http://127.0.0.1:8000/v1/models", timeout=10)
print("Status:", r.status_code)
print(r.text[:500])

Status: 200
{"object":"list","data":[{"id":"LangAGI-Lab/camel","object":"model","created":1770126571,"owned_by":"vllm","root":"LangAGI-Lab/camel","parent":null,"max_model_len":8192,"permission":[{"id":"modelperm-93202fda5b567cc8","object":"model_permission","created":1770126571,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}


## Define requirements

In [2]:
from pathlib import Path

DATA_PATH = Path("data/Patient_PSi_CM_Dataset_Planning_Resistance.json")
CLIENT_PROMPT_PATH = Path("prompts/client.txt")
CRITIC_PROMPT_PATH = Path("prompts/trust_critic.txt")
MOD_PROMPT_PATH = Path("prompts/moderator.txt")

In [3]:
VLLM_SERVER = "http://127.0.0.1:8000/v1".strip()
CAMEL_MODEL_ID = "LangAGI-Lab/camel"

In [4]:
# models for your simulation components
client_model = "gpt-4o-mini"
critic_model = "gpt-4o"
moderator_model = "gpt-4o"

In [5]:
from helpers import (
    load_text,
    load_patients,
    pick_patient,
    normalize_patient,
    render_template,
    parse_trust_score,
    parse_yes_no,
    format_dialogue,
    next_phase,
    trust_eval_interval,
    print_last_turn,
    cactus_to_intake_reason,
    trim_camel_history
)

from llm import call_llm
from llm import call_llm_messages

/home/luk/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
CASE_ID = "2-1"

# Load prompts
client_template = load_text(CLIENT_PROMPT_PATH)
critic_template = load_text(CRITIC_PROMPT_PATH)
mod_template = load_text(MOD_PROMPT_PATH)

# 1) Load PSI patients and select patient
patients = load_patients(data_path=DATA_PATH)
patient = pick_patient(patients, patient_id=CASE_ID)
p = normalize_patient(patient)

In [7]:
PSI_JSON_PATH = "data/Patient_PSi_CM_Dataset_Planning_Resistance.json"
PSI_TO_CACTUS_SYSTEM_PROMPT_PATH = "prompts/psi_to_cactus_system.txt"
PSI_TO_CACTUS_MODEL = "gpt-4o-mini"

In [8]:
from psi_to_cactus import convert_psi_file_case_id_to_cactus

# 2) Convert PSI -> CACTUS (intake-style object)
cactus_obj = convert_psi_file_case_id_to_cactus(
    psi_json_path=PSI_JSON_PATH,
    case_id=CASE_ID,
    system_prompt_path=PSI_TO_CACTUS_SYSTEM_PROMPT_PATH,
    call_llm_fn=call_llm,
    model=PSI_TO_CACTUS_MODEL,
    temperature=0.0,
)

In [ ]:
# import json
# from psi_to_cactus import convert_psi_file_all_cases_to_cactus

# results = convert_psi_file_all_cases_to_cactus(
#     psi_json_path=PSI_JSON_PATH,
#     system_prompt_path=PSI_TO_CACTUS_SYSTEM_PROMPT_PATH,
#     call_llm_fn=call_llm,
#     model=PSI_TO_CACTUS_MODEL,
#     temperature=0.0
# )

# with open("data/cactus_all_cases.json", "w", encoding="utf-8") as f:
#     json.dump(results, f, ensure_ascii=False, indent=2)

In [9]:
from camel_agent import CamelCounselingSession, CounselorAgent, RESPONSE_PROMPT

# 3) Initialize CAMEL therapist session
sess = CamelCounselingSession(
    vllm_server=VLLM_SERVER,
    model_id=CAMEL_MODEL_ID,
    temperature=0.7,
)
intake_form, reason = cactus_to_intake_reason(sess, cactus_obj)

In [10]:
# # Optional: enrich 'p' fields used by your client template
# # (This keeps your client simulation consistent with CACTUS-derived information.)
# intake = cactus_obj.get("intake_form", {}) or {}
# ci = intake.get("client_info", {}) or {}
# p["name"] = ci.get("name", p.get("name", "Client"))
# p["history"] = "\n".join(intake.get("past_history", []) or []) if isinstance(intake.get("past_history"), list) else str(intake.get("past_history", "") or "")
# p["situation"] = cactus_obj.get("thought", p.get("situation", ""))

In [11]:
from typing import Any, Dict, List, Optional

# how often to run trust critic
interval = trust_eval_interval(p.get("resistance_level"))

# phase state
phase = "trust_building"
openness = 1
trust_level = 1

In [12]:
# convo format is your simulator format: assistant=user roles (therapist/client)
convo: List[Dict[str, str]] = []
turns: List[Dict[str, Any]] = []

In [13]:
# -----------------------
# Therapist starts (Turn 1 therapist message)
# -----------------------
therapist_first = "Hi, it’s nice to meet you. What brings you to therapy today?"
convo.append({"role": "assistant", "content": therapist_first})
current_therapist_reply = therapist_first

In [ ]:
MAX_TURNS = 30

# Used to produce the first therapist reply after planning, without duplicating client message
first_reply_generated = False

for t in range(1, MAX_TURNS + 1):
    # -------- client responds to current therapist prompt
    p["trust_level"] = trust_level
    p["stage_therapy"] = phase

    client_system = render_template(client_template, p)
    client_user = (
        "Conversation so far:\n"
        f"{format_dialogue(convo, last_n=24)}\n\n"
        "Respond as the client to the therapist's latest message."
    )

    client_text = call_llm_messages(
        [{"role": "system", "content": client_system},
         {"role": "user", "content": client_user}],
        temperature=0.7,
        model=client_model,
    )
    convo.append({"role": "user", "content": client_text})

    # -------- critic (openness) — evaluate every N turns
    should_eval = (t % interval == 0)
    if should_eval:
        critic_system = render_template(
            critic_template,
            {"dialogue_context": format_dialogue(convo, last_n=16)},
        )
        critic_text = call_llm_messages(
            [{"role": "system", "content": critic_system}],
            model=critic_model,
        )
        score = parse_trust_score(critic_text)
        if score is not None:
            openness = score
    else:
        critic_text = None

    # phase progression — ONLY when critic ran
    if should_eval:
        phase = next_phase(phase, openness)

    trust_level = openness

    # -------- moderator end? (must decide BEFORE generating next therapist reply)
    mod_system = render_template(mod_template, {"conversation": format_dialogue(convo, last_n=24)})
    mod_text = call_llm_messages([{"role": "system", "content": mod_system}], model=moderator_model)
    end_flag = parse_yes_no(mod_text)
    if end_flag is None:
        end_flag = False

    # -------- save ONE combined turn record (therapist prompt + client response)
    turns.append({
        "turn_id": t,
        "phase_for_next_turn": phase,
        "openness": openness,
        "therapist": current_therapist_reply,
        "client": client_text,
        "critic_raw": critic_text,
        "moderator_raw": mod_text,
        "end_session": end_flag,
    })

    print_last_turn(convo, t)

    # If moderator ends after client, stop here (NO new therapist reply)
    if end_flag:
        break

    # -----------------------
    # Therapist generates the NEXT prompt (CAMEL)
    # -----------------------
    if not first_reply_generated:
        # First time: build CAMEL plan using the first client message
        sess.start(intake_form=intake_form, reason=reason, first_client_message=client_text)

        # Trim ONLY therapist-side history before generating the reply
        sess.history = trim_camel_history(sess.history, keep_last=25)        

        # Now generate the therapist response to that same client message
        # WITHOUT sending the client_text again (prevents duplication)
        counselor = CounselorAgent(
            sess.vllm_server, 
            sess.model_id, 
            sess.cbt_plan,  
            RESPONSE_PROMPT,  
        )
        therapist_reply = counselor.next_utterance(intake_form, reason, sess.history)
        sess.history.append({"role": "Counselor", "message": therapist_reply})

        # Trim again after appending reply (keeps it bounded forever)
        sess.history = trim_camel_history(sess.history, keep_last=25) 

        first_reply_generated = True
    else:
        # Normal turns: before calling sess.step(), trim therapist history
        sess.history = trim_camel_history(sess.history, keep_last=25)
    
        therapist_reply = sess.step(client_text)
    
        # sess.step() appends client+reply internally; trim again to keep bounded
        sess.history = trim_camel_history(sess.history, keep_last=25)

    convo.append({"role": "assistant", "content": therapist_reply})
    current_therapist_reply = therapist_reply


======= TURN 1 =======
Therapist: Hi, it’s nice to meet you. What brings you to therapy today?
Client: Well, I guess I’m just feeling really overwhelmed with everything that's been going on, especially with my family. There's been this ongoing tension, particularly around planning my wedding and the guest list. It's like no matter what I do, I can't seem to make everyone happy. I feel like I'm stuck in this cycle of trying to get their approval, but it’s just causing me more frustration and anxiety. I don’t know, maybe I’m just not handling it right or something.


======= TURN 2 =======
Therapist: It sounds like you're dealing with a lot of pressure from different directions. Can you tell me more about what specifically is causing you the most stress?
Client: I mean, it's just the whole situation with my parents, really. We had this conversation about the wedding list, and they have such strong opinions on who should be invited. I feel like I can't even express my own thoughts withou

In [ ]:
import json

final_output = {
    "patient_id": str(p.get("id", "")),
    "patient_name": str(p.get("name", "")),
    "turns": turns
}

save_output_path = f"outputs/{final_output['patient_name']}_{final_output['patient_id']}.json"

Path(save_output_path).write_text(json.dumps(final_output, ensure_ascii=False, indent=2), encoding="utf-8")
print(f"Saved: {save_output_path}")